### Problem Statement

This data was extracted from the census bureau database found at

http://www.census.gov/ftp/pub/DES/www/welcome.html

Donor: Ronny Kohavi and Barry Becker, Data Mining and Visualization Silicon Graphics.

e-mail: ronnyk@sgi.com for questions.

Split into train-test using MLC++ GenCVFiles (2/3, 1/3 random).

48842 instances, mix of continuous and discrete (train=32561, test=16281)

45222 if instances with unknown values are removed (train=30162, test=15060)

Duplicate or conflicting instances : 6

Class probabilities for adult.all file

Probability for the label '>50K' : 23.93% / 24.78% (without unknowns)

Probability for the label '<=50K' : 76.07% / 75.22% (without unknowns)

Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions:

((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0)) 

Prediction task is to determine whether a person makes over 50K a year. Conversion of original data as follows:

1. Discretized a gross income into two ranges with threshold 50,000.
2. Convert U.S. to US to avoid periods.
3. Convert Unknown to "?"
4. Run MLC++ GenCVFiles to generate data,test.

##### Description of fnlwgt (final weight) 

The weights on the CPS files are controlled to independent estimates of the civilian noninstitutional population of the US. These are prepared monthly for us by Population Division here at the Census Bureau. We use 3 sets of controls.

These are:

1. A single cell estimate of the population 16+ for each state.
2. Controls for Hispanic Origin by age and sex.
3. Controls by Race, age and sex.

We use all three sets of controls in our weighting program and "rake" through them 6 times so that by the end we come back to all the controls we used. The term estimate refers to population totals derived from CPS by creating "weighted
tallies" of any specified socio-economic characteristics of the population. People with similar demographic characteristics should have similar weights. There is one important caveat to remember about this statement. That is that since the CPS sample is actually a collection of 51 state samples, each with its own probability of selection, the statement
only applies within state.

##### Dataset Link

https://archive.ics.uci.edu/ml/machine-learning-databases/adult/



In [108]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import Imputer
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
import xgboost

from sklearn.metrics import accuracy_score


#### Problem 1:

Prediction task is to determine whether a person makes over 50K a year.


In [142]:
adult_df = pd.read_csv(r'F:\Data Science Master\Data\adult.data', header = None, delimiter=' *, *', engine='python')

In [143]:
adult_df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
                    'marital_status', 'occupation', 'relationship',
                    'race', 'sex', 'capital_gain', 'capital_loss',
                    'hours_per_week', 'native_country', 'income']

In [144]:
adult_df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [145]:
adult_df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64

In [146]:
adult_df['capital_gain'].value_counts()

0        29849
15024      347
7688       284
7298       246
99999      159
         ...  
4931         1
1455         1
6097         1
22040        1
1111         1
Name: capital_gain, Length: 119, dtype: int64

In [147]:
adult_df['capital_loss'].value_counts()

0       31042
1902      202
1977      168
1887      159
1848       51
        ...  
1411        1
1539        1
2472        1
1944        1
2201        1
Name: capital_loss, Length: 92, dtype: int64

In [148]:
for value in ['workclass', 'education',
          'marital_status', 'occupation',
          'relationship','race', 'sex',
          'native_country', 'income']:
    print(value,":", sum(adult_df[value] == '?'))

workclass : 1836
education : 0
marital_status : 0
occupation : 1843
relationship : 0
race : 0
sex : 0
native_country : 583
income : 0


In [149]:
adult_df_new = adult_df

In [150]:
adult_df_new.describe(include= 'all')

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
count,32561.000000,32561,3.256100e+04,32561,32561.000000,32561,32561,32561,32561,32561,32561.000000,32561.000000,32561.000000,32561,32561
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,22696,NaN,10501,NaN,14976,4140,13193,27816,21790,NaN,NaN,NaN,29170,24720
mean,38.581647,NaN,1.897784e+05,NaN,10.080679,NaN,NaN,NaN,NaN,NaN,1077.648844,87.303830,40.437456,NaN,NaN
std,13.640433,NaN,1.055500e+05,NaN,2.572720,NaN,NaN,NaN,NaN,NaN,7385.292085,402.960219,12.347429,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.178270e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.783560e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.370510e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


#### Data Imputation

In [151]:
for value in ['workclass', 'education',
          'marital_status', 'occupation',
          'relationship','race', 'sex',
          'native_country', 'income']:
    adult_df_new[value].replace(['?'], [adult_df_new.describe(include='all')[value][2]],
                                inplace=True)
    
adult_df_new.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### One-Hot Encoder

In [152]:
le = preprocessing.LabelEncoder()
workclass_cat = le.fit_transform(adult_df.workclass)
education_cat = le.fit_transform(adult_df.education)
marital_cat   = le.fit_transform(adult_df.marital_status)
occupation_cat = le.fit_transform(adult_df.occupation)
relationship_cat = le.fit_transform(adult_df.relationship)
race_cat = le.fit_transform(adult_df.race)
sex_cat = le.fit_transform(adult_df.sex)
native_country_cat = le.fit_transform(adult_df.native_country)

In [153]:
#initialize the encoded categorical columns
adult_df_new['workclass_cat'] = workclass_cat
adult_df_new['education_cat'] = education_cat
adult_df_new['marital_cat'] = marital_cat
adult_df_new['occupation_cat'] = occupation_cat
adult_df_new['relationship_cat'] = relationship_cat
adult_df_new['race_cat'] = race_cat
adult_df_new['sex_cat'] = sex_cat
adult_df_new['native_country_cat'] = native_country_cat

adult_df_new.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,...,native_country,income,workclass_cat,education_cat,marital_cat,occupation_cat,relationship_cat,race_cat,sex_cat,native_country_cat
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,...,United-States,<=50K,6,9,4,0,1,4,1,38
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,United-States,<=50K,5,9,2,3,0,4,1,38
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,...,United-States,<=50K,3,11,0,5,1,4,1,38
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,...,United-States,<=50K,3,1,2,5,0,2,1,38
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,...,Cuba,<=50K,3,9,2,9,5,2,0,4


In [154]:
#drop the old categorical columns from dataframe
dummy_fields = ['workclass', 'education', 'marital_status', 
                  'occupation', 'relationship', 'race',
                  'sex', 'native_country']
adult_df_new = adult_df_new.drop(dummy_fields, axis = 1)
adult_df_new.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,income,workclass_cat,education_cat,marital_cat,occupation_cat,relationship_cat,race_cat,sex_cat,native_country_cat
0,39,77516,13,2174,0,40,<=50K,6,9,4,0,1,4,1,38
1,50,83311,13,0,0,13,<=50K,5,9,2,3,0,4,1,38
2,38,215646,9,0,0,40,<=50K,3,11,0,5,1,4,1,38
3,53,234721,7,0,0,40,<=50K,3,1,2,5,0,2,1,38
4,28,338409,13,0,0,40,<=50K,3,9,2,9,5,2,0,4


In [155]:
adult_df_new.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,income,workclass_cat,education_cat,marital_cat,occupation_cat,relationship_cat,race_cat,sex_cat,native_country_cat
0,39,77516,13,2174,0,40,<=50K,6,9,4,0,1,4,1,38
1,50,83311,13,0,0,13,<=50K,5,9,2,3,0,4,1,38
2,38,215646,9,0,0,40,<=50K,3,11,0,5,1,4,1,38
3,53,234721,7,0,0,40,<=50K,3,1,2,5,0,2,1,38
4,28,338409,13,0,0,40,<=50K,3,9,2,9,5,2,0,4


In [156]:
adult_df_new = adult_df_new.reindex(['age', 'workclass_cat', 'fnlwgt', 'education_cat',
                                    'education_num', 'marital_cat', 'occupation_cat',
                                    'relationship_cat', 'race_cat', 'sex_cat', 'capital_gain',
                                    'capital_loss', 'hours_per_week', 'native_country_cat', 
                                    'income'], axis= 1)
 
adult_df_new.head(1)

,age,workclass_cat,fnlwgt,education_cat,education_num,marital_cat,occupation_cat,relationship_cat,race_cat,sex_cat,capital_gain,capital_loss,hours_per_week,native_country_cat,income
0,39,6,77516,9,13,4,0,1,4,1,2174,0,40,38,<=50K


##### Building the model

In [157]:
x = adult_df_new.values[:,:14]
y = adult_df_new.values[:,14]

In [158]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.30, random_state = 10)

In [159]:
classifier = xgboost.XGBClassifier()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

y_pred

array(['<=50K', '<=50K', '>50K', ..., '<=50K', '<=50K', '>50K'],
      dtype=object)

In [160]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
accuracies

array([0.86973684, 0.85526316, 0.85789474, 0.87017544, 0.86315789,
       0.86353664, 0.8490566 , 0.87093942, 0.87225637, 0.8713784 ])

In [161]:
print(accuracies.mean())
print(accuracies.std())

0.8643395500709833
0.007605879189660385


In [162]:
def _build_df_from_confusion_matrix(confusion_matrix, as_fractions=False):
    if as_fractions:
        x = np.array(confusion_matrix)
        x = np.apply_along_axis(
            lambda row: [
                row[0] / (row[0] + row[1]),
                row[1] / (row[0] + row[1])
            ],
            1,
            x
        )
    else:
        x = confusion_matrix
    df = pd.DataFrame(
        x,
        index=["<= 50K", "> 50K"],
        columns=["<= 50K", "> 50K"]
    )
    df.index.names = ["Actual"]
    df.columns.names = ["Predicted"]
    return df

In [163]:
display(_build_df_from_confusion_matrix(cm, as_fractions=False))

Predicted,<= 50K,> 50K
Actual,,
<= 50K,7051,372
> 50K,989,1357


#### Problem 2:

Which factors are important

In [131]:
adult_df_new.columns

Index(['age', 'workclass_cat', 'fnlwgt', 'education_cat', 'education_num',
       'marital_cat', 'occupation_cat', 'relationship_cat', 'race_cat',
       'sex_cat', 'capital_gain', 'capital_loss', 'hours_per_week',
       'native_country_cat', 'income'],
      dtype='object')

In [132]:
adult_df_new.head()

,age,workclass_cat,fnlwgt,education_cat,education_num,marital_cat,occupation_cat,relationship_cat,race_cat,sex_cat,capital_gain,capital_loss,hours_per_week,native_country_cat,income
0,39,6,77516,9,13,4,0,1,4,1,2174,0,40,38,<=50K
1,50,5,83311,9,13,2,3,0,4,1,0,0,13,38,<=50K
2,38,3,215646,11,9,0,5,1,4,1,0,0,40,38,<=50K
3,53,3,234721,1,7,2,5,0,2,1,0,0,40,38,<=50K
4,28,3,338409,9,13,2,9,5,2,0,0,0,40,4,<=50K


In [133]:
adult_df_sec = adult_df_new
adult_df_sec.drop("capital_gain", axis=1, inplace=True,)
adult_df_sec.drop("capital_loss", axis=1, inplace=True,)

In [134]:
adult_df_sec.head()

,age,workclass_cat,fnlwgt,education_cat,education_num,marital_cat,occupation_cat,relationship_cat,race_cat,sex_cat,hours_per_week,native_country_cat,income
0,39,6,77516,9,13,4,0,1,4,1,40,38,<=50K
1,50,5,83311,9,13,2,3,0,4,1,13,38,<=50K
2,38,3,215646,11,9,0,5,1,4,1,40,38,<=50K
3,53,3,234721,1,7,2,5,0,2,1,40,38,<=50K
4,28,3,338409,9,13,2,9,5,2,0,40,4,<=50K


In [135]:
x = adult_df_sec.values[:,:12]
y = adult_df_sec.values[:,12]
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.30, random_state = 10)
classifier_new = xgboost.XGBClassifier()
classifier_new.fit(X_train, y_train)

# Predicting the Test set results
y_pred_new = classifier_new.predict(X_test)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm_new = confusion_matrix(y_test, y_pred_new)

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies_new = cross_val_score(estimator = classifier_new, X = X_train, y = y_train, cv = 10)

print(accuracies_new.mean())
print(accuracies_new.std())

0.8396373074467389
0.007572817261782365


In [136]:
display(_build_df_from_confusion_matrix(cm_new, as_fractions=False))

Predicted,<= 50K,> 50K
Actual,,
<= 50K,6891,532
> 50K,1019,1327


In the second set of prediction, I tried to check whether capital gain and capital loss features are significant or not. So in the second set, those features were dropped and verified. It is observed that without capital gain and capital loss features, the accuracies got dropped. So it looks like both the features are significant.

#### Problem 3:

Which algorithms are best for this dataset

##### Random Forest

In [140]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)

Y_pred_rf = random_forest.predict(X_test)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)

##### XGBoost

In [164]:
acc_xgBoost = round(classifier.score(X_train,y_train) * 100,2)

In [168]:
results = pd.DataFrame({
    'Model': [ 'Random Forest', 'XGBoost'],
    'Score': [acc_random_forest, acc_xgBoost]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head(9)

,Model
Score,
99.99,Random Forest
86.79,XGBoost


It is observed that Random Forest works better than XGBoost for this dataset